In [1]:
import datasets
ds = datasets.load_from_disk("./taco_cleaned")
ds

/Users/federico.cassano/.pyenv/versions/3.9.19/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['question', 'solutions', 'starter_code', 'input_output', 'difficulty', 'raw_tags', 'name', 'source', 'tags', 'skill_types', 'url', 'Expected Auxiliary Space', 'time_limit', 'date', 'picture_num', 'memory_limit', 'Expected Time Complexity'],
    num_rows: 9029
})

In [2]:
# figure out format of tests
import json
import sys
sys.set_int_max_str_digits(0)
keys = set()
types = set()
has_fn_name = 0

for ex in ds:
    eio = ex["input_output"]
    eio = json.loads(eio)
    types.add(type(eio))
    ks = eio.keys()
    if "fn_name" in ks:
        has_fn_name += 1
    for k in ks:
        keys.add(k)

print(types)
print(keys)
print(has_fn_name)

{<class 'dict'>}
{'fn_name', 'inputs', 'outputs'}
544


In [2]:
9000 * 5

# MinHash + Locality Sentitive Hasking, Jaccard Similarity 0.5
# pick 5 dedup
# 1 with original prompt, 4 LLM to rephrase the prompt

45000

In [4]:
ds = ds.shuffle()

In [7]:
import json
from tqdm import tqdm
from codeprm.execution import exec_io_test

subset = ds.select(range(10))
for i, ex in enumerate(subset):
    print(f"++ RUNNING {i}")
    for sol in tqdm(ex["solutions"][:5]):
        tests = json.loads(ex["input_output"])
        inputs = tests["inputs"]
        outputs = tests["outputs"]
        print(exec_io_test(sol, inputs, outputs))

++ RUNNING 0


 20%|████████████████████████████████████████████████████████▏                                                                                                                                                                                                                                | 1/5 [00:01<00:05,  1.45s/it]

(False, "['3\\n2\\n5 5\\n2 3\\n3\\n3 5\\n1 3\\n2 3\\n3\\n3 3\\n4 4\\n5 5\\n'] expected '2 1\\n-1\\n1 2 2\\n' but got '2 1\\n-1\\n1 1 2\\n'\n['3\\n2\\n3 5\\n2 3\\n3\\n3 5\\n1 5\\n2 3\\n3\\n3 3\\n4 4\\n5 4\\n'] expected '-1\\n-1\\n1 2 2\\n' but got '-1\\n-1\\n1 1 2\\n'\n['3\\n2\\n5 5\\n2 3\\n3\\n3 5\\n1 3\\n0 3\\n3\\n3 3\\n4 4\\n5 5\\n'] expected '2 1\\n-1\\n1 2 2\\n' but got '2 1\\n-1\\n1 1 2\\n'\n['3\\n2\\n5 5\\n2 3\\n3\\n3 5\\n1 5\\n2 3\\n3\\n3 3\\n4 4\\n5 5\\n'] expected '2 1\\n-1\\n1 2 2\\n' but got '2 1\\n-1\\n1 1 2\\n'\n['3\\n2\\n5 5\\n2 3\\n3\\n3 5\\n1 5\\n2 3\\n3\\n3 3\\n4 4\\n5 4\\n'] expected '2 1\\n-1\\n1 2 2\\n' but got '2 1\\n-1\\n1 1 2\\n'\n['3\\n2\\n3 5\\n2 3\\n3\\n3 5\\n1 5\\n0 3\\n3\\n3 3\\n4 4\\n5 4\\n'] expected '-1\\n-1\\n1 2 2\\n' but got '-1\\n-1\\n1 1 2\\n'\n['3\\n2\\n3 10\\n2 3\\n3\\n3 5\\n1 5\\n0 3\\n3\\n3 3\\n4 4\\n5 4\\n'] expected '-1\\n-1\\n1 2 2\\n' but got '-1\\n-1\\n1 1 2\\n'\n['3\\n2\\n4 10\\n2 3\\n3\\n3 5\\n1 5\\n0 3\\n3\\n3 3\\n4 4\\n5 4\\n'] expected 

 40%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                                        | 2/5 [00:03<00:04,  1.53s/it]

(False, "['3\\n2\\n5 5\\n2 3\\n3\\n3 5\\n2 3\\n2 3\\n3\\n3 3\\n4 4\\n5 5\\n'] expected '2 1 \\n-1\\n1 1 2 \\n' but got '2 1\\n-1\\n1 2 2\\n'\n")


...

(False, "['3 9\\n2 1\\n2 2\\n8 5'] expected '-2.98826040\\n' but got '257.4572624074256\\n'\n['3 8\\n1 1\\n12 2\\n8 1'] expected '-2.15756505\\n' but got '203.21428823597392\\n'\n['3 9\\n2 1\\n2 2\\n8 1'] expected '-3.00000000\\n' but got '3.0\\n'\n['3 9\\n2 1\\n2 2\\n13 1'] expected '-4.74366233\\n' but got '259.2116787739863\\n'\n['3 9\\n2 1\\n2 2\\n13 0'] expected '-4.76640990\\n' but got '259.2344836502092\\n'\n['4 8\\n-4 0\\n0 -5\\n2 -3\\n6 -6'] expected '6.01264633\\n' but got '207.07457615568953\\n'\n['4 4\\n0 -2\\n-6 -7\\n1 -3\\n1 -4'] expected '2.14387606\\n' but got '52.746653994964895\\n'\n['3 1\\n-1 -1\\n0 1\\n11 5'] expected '-1.72729522\\n' but got '1.4142974355881779\\n'\n")


 67%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                             | 2/3 [00:05<00:02,  2.65s/it]

(True, '')


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.64s/it]

(True, '')
